In [1]:
# package imports
import base64
from dotenv import load_dotenv
import json
import math
import numpy as np
import os
import pandas as pd
import requests

# relative imports
import auth.spotify as spotify

In [2]:
# read environment variables
load_dotenv()
# client id and secret
SPOTIFY_API_CLIENT_ID = os.getenv('SPOTIFY_API_CLIENT_ID')
SPOTIFY_API_CLIENT_SECRET = os.getenv('SPOTIFY_API_CLIENT_SECRET')
# refresh token
SPOTIFY_REFRESH_TOKEN = os.getenv('SPOTIFY_REFRESH_TOKEN')
# api urls
SPOTIFY_API_BASE_URL = os.getenv('SPOTIFY_API_BASE_URL')
SPOTIFY_API_TOKEN_URL = os.getenv('SPOTIFY_API_TOKEN_URL')
# redirect uri
SPOTIFY_API_REDIRECT_URI = os.getenv('SPOTIFY_API_REDIRECT_URI')
# api paths
TRACK_AUDIO_FEATURES_ENDPOINT = os.getenv('TRACK_AUDIO_FEATURES_ENDPOINT')
USER_TOP_TRACKS_ENDPOINT = os.getenv('USER_TOP_TRACKS_ENDPOINT')

In [3]:
res = requests.get(f"{SPOTIFY_API_BASE_URL}{USER_TOP_TRACKS_ENDPOINT}", headers = {
    "Authorization": f"Bearer {spotify.get_access_token()}"
}).json()

# print(json.dumps(res['items'][0], indent=2))

In [27]:
top_tracks = []
for item in res['items']:
    artists = [artist['name'] for artist in item['artists']]
    total_seconds = math.floor(item['duration_ms'] / 1000)
    leftover_seconds = total_seconds % 60
    duration = f"{total_seconds // 60}:{'0{}'.format(leftover_seconds) if leftover_seconds < 10 else leftover_seconds}"
    track = [
        item['id'],
        item['name'],
        ', '.join(artists),
        duration,
        item['album']['id'],
        item['album']['name'],
        item['album']['images'][0]
    ]
    top_tracks.append(track)

In [28]:
def fetch_track_audio_features(track_id):
    res = requests.get(f"{SPOTIFY_API_BASE_URL}{TRACK_AUDIO_FEATURES_ENDPOINT}{track_id}", headers = {
        "Authorization": f"Bearer {spotify.get_access_token()}"
    }).json()
    return res

for track in top_tracks:
    audio_features = fetch_track_audio_features(track[0])
    track += [audio_features['danceability'], audio_features['energy'], audio_features['key'],
              audio_features['loudness'], audio_features['mode'], audio_features['speechiness'],
              audio_features['acousticness'], audio_features['instrumentalness'], audio_features['liveness'],
              audio_features['valence'], audio_features['tempo']]
    

In [29]:
# print(len(top_tracks[0]))
# tracks_df = pd.DataFrame(top_tracks, columns=[
#     'Track ID', 'Track Name', 'Artists', 'Duration', 'Album ID', 'Album Name', 'Image',
#     'Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness',
#     'Liveness', 'Valence', 'Tempo'
# ])
# display(tracks_df)

18


,Track ID,Track Name,Artists,Duration,Album ID,Album Name,Image,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,5SMUyfT2P1kpKqWbkAResU,Decatur,6LACK,2:57,39JZhYotu5zFYJ8jw1D8bF,Since I Have A Lover,"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.506,0.6760,1,-6.168,1,0.3310,0.38700,0.285000,0.3830,0.4280,195.843
1,0TfMitxzrMd2NTLxNXL2Hu,Georgia Peach,Quinn XCII,2:21,68ht4TaMromCg26Fu3fBB0,The People's Champ (Extended Version),"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.774,0.6980,11,-5.448,0,0.0479,0.01130,0.000065,0.0676,0.8670,97.002
2,3mwFncaI2HBczQ92GP9MQF,Lady Writer,Dire Straits,3:49,6HEOoO8aHq5M88OA4teVY1,Communiqué,"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.633,0.7980,1,-9.313,0,0.0281,0.16600,0.617000,0.0812,0.9630,147.645
3,1c03f9YZmO792D7hRjZMYi,Hands On You,Austin George,2:35,7AwODiyS5ghh0uhXt4v2tj,Hands On You,"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.616,0.5570,11,-5.871,0,0.0413,0.35400,0.000002,0.3450,0.6260,159.910
4,18atM21acJCu9bH0YCsYG0,Believe,"NORRA, lostinspace",2:13,3VLYzHg6S1JngbXYapwMF3,Believe,"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.648,0.6790,10,-6.240,1,0.1510,0.06450,0.012300,0.1000,0.1100,119.935
5,6FBPOJLxUZEair6x4kLDhf,Any Colour You Like,Pink Floyd,3:26,4LH4d3cOWNNsVw41Gqt2kv,The Dark Side of the Moon,"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.277,0.6720,0,-12.582,1,0.0901,0.19700,0.936000,0.4230,0.5050,150.587
6,7lWdyj9RrMsEN6F0Dl0irE,Assumptions - Kaytranada Edit,"Sam Gellaitry, KAYTRANADA",4:48,5YTiQmOzu1m293UgtWNJSR,Assumptions (Kaytranada Edit),"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.722,0.7800,5,-8.623,0,0.0753,0.00659,0.047000,0.0545,0.2000,119.997
7,1hJrWWK74fKL7eeV3CFOvF,Sweet Virginia,The Rolling Stones,4:26,5U4dnRZsfW8NmwBBkELFPh,Exile On Main Street (2010 Re-Mastered),"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.477,0.7400,9,-5.397,1,0.0446,0.50200,0.000001,0.2630,0.5600,107.384
8,7aFfZ8sqPgMDxBdoV9mclI,Coastal Cat,Tez Cadey,4:52,6PEGEok2ASsFO0yJMhsUZ2,Coastal Cat,"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.763,0.3810,9,-12.992,0,0.0746,0.02550,0.837000,0.0837,0.1790,118.994
9,6pnwfWyaWjQiHCKTiZLItr,Shine On You Crazy Diamond (Pts. 1-5),Pink Floyd,13:31,0bCAjiUamIFqKJsekOYuRw,Wish You Were Here,"{'height': 640, 'url': 'https://i.scdn.co/imag...",0.264,0.2940,7,-11.941,0,0.0280,0.77200,0.686000,0.1070,0.0402,137.937
